# Lending Tree Loan Status - Hyperopt

# Set Up Environment for Spark

## Colab

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Spark/

/content/drive/MyDrive/Spark


In [ ]:
# Set up environment for Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.3.0-bin-hadoop3.tgz

In [ ]:
# Set your spark folder to your system path environment. 
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/drive/MyDrive/Spark/spark-3.3.0-bin-hadoop3'

In [ ]:
# Install findspark using pip
!pip install -q findspark
!pip install -U pyspark==3.3
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 63.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2ac9238506d28218a2c824c8c7bc44df580e10a8b1f116a72abe92b2f2219e0d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# Pyspark Session for Colab
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('Colab')\
        .config('spark.driver.memory', '24g')\
        .config('spark.executor.pyspark.memory', '18g')\
        .config('spark.executor.cores', '4')\
        .config('spark.python.worker.memory', '18g')\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'True')\
        .config('spark.sql.debug.maxToStringFields', '1000')\
        .config('spark.sql.autoBroadcastJoinThreshold', '-1')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

## Paperspace Gradient

In [ ]:
# Install Java JRE/JDK in terminal
apt update
apt install default-jre
apt install default-jdk

In [ ]:
# Install findspark using pip
!pip install -q findspark
!pip install -U pyspark==3.3
import findspark
findspark.init()

In [ ]:
# Pyspark Session for Paperspace with more RAM & CPU cores available
# driver.memory=35g & executor.pyspark.memory=28 for some
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('Paperspace')\
        .config('spark.driver.memory', '38g')\
        .config('spark.executor.pyspark.memory', '32g')\
        .config('spark.executor.cores', '4')\
        .config('spark.python.worker.memory', '32g')\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'True')\
        .config('spark.sql.debug.maxToStringFields', '1000')\
        .config('spark.sql.autoBroadcastJoinThreshold', '-1')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/16 15:32:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Remove warnings
spark.sparkContext.setLogLevel('ERROR')

# Install & Import Packages and Set Seed

In [ ]:
#!pip install --upgrade mlflow 
!pip install git+https://github.com/mlflow/mlflow@master
!pip install hyperopt
import random
import warnings
import numpy as np
from pyspark.sql.functions import col, round
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import time
from datetime import datetime, timedelta
from timeit import default_timer as timer
try:
  import mlflow.pyspark.ml
  mlflow.pyspark.ml.autolog()
except:
  print(f'Your version of MLflow ({mlflow.__version__}) does not support pyspark.ml for autologging. To use autologging, upgrade your MLflow client version or use Databricks Runtime for ML 8.3 or above.')
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mlflow/mlflow (to revision master) to /tmp/pip-req-build-mz33pnsf
  Running command git clone -q https://github.com/mlflow/mlflow /tmp/pip-req-build-mz33pnsf
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 2.1 MB/s 
     |████████████████████████████████| 182 kB 20.1 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 59.9 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 78 kB 4.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 140 kB 58.8 MB/s 
     |██████████

In [ ]:
# Set seed 
seed_value = 42
os.environ['SparkML_HPO'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

# Upsampling

## Read Data and View Schema

In [ ]:
trainDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/trainDF_US.csv',
                         header=True, inferSchema=True)
trainDF.cache()
print('\nTrain Schema')
trainDF.printSchema()

testDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/testDF_US.csv',
                        header=True, inferSchema=True)
testDF.cache()
print('\nTest Schema')
testDF.printSchema()


Train Schema
root
 |-- loan_amnt: integer (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- collections_12_mths_ex_med: double (nullable = true)
 |-- acc_open_past_24mths: double (nullable = true)
 |-- bc_open_to_buy: double (nullable = true)
 |-- chargeoff_within_12_mths: double (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- mths_since_recent_bc: double (nullable = true)
 |-- num_bc_sats: double (nullable = true)
 |-- num_bc_tl: double (nullable = true)
 |-- num_sats: double (nulla

## Set up Vector Assembler, Scalers and Evaluators

In [ ]:
# Define features and label for train data
features = trainDF.columns[0: len(trainDF.columns) - 1]
trainDF = trainDF.select(col('loan_status').alias('label'), *features)

In [ ]:
# VectorAssembler 
vecAssembler = VectorAssembler(inputCols=features, 
                               outputCol='unscaledFeatures', 
                               handleInvalid='skip')  

# Transform train data
trainDF = vecAssembler.transform(trainDF) 

In [ ]:
# Define features and label for test data 
features = testDF.columns[0: len(testDF.columns) - 1]
testDF = testDF.select(col('loan_status').alias('label'), *features)

# Transform test data
testDF = vecAssembler.transform(testDF)  

In [ ]:
# MinMaxScaler
mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                        outputCol='scaledFeatures',
                        min=0, max=1)

In [ ]:
# Standard Scaler
stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                           outputCol='scaledFeatures', 
                           withStd=True, 
                           withMean=False)

In [ ]:
# Define model evaluation methods
evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                metricName='areaUnderROC')

evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                 metricName='f1')

evaluator_acc = MulticlassClassificationEvaluator(labelCol='label', 
                                                  metricName='accuracy')

In [ ]:
# Change directory to write mlruns
%cd /content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/

/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt


## Logistic Regression


### AUROC Score 

#### 100 Trials

In [ ]:
def train_model(regParam, elasticNetParam): 
  
  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # MinMaxScaler
    mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                            outputCol='scaledFeatures',
                            min=0, max=1)
    
    # Define model
    lr = LogisticRegression(labelCol='label', 
                            featuresCol='scaledFeatures',
                            family='binomial',                             
                            regParam=regParam,
                            elasticNetParam=elasticNetParam,
                            standardization=True)
    
    print('\nTrial Parameters:')
    print('regParam:', lr.getRegParam())
    print('ElasticNet:', lr.getElasticNetParam())
    print('maxIter:', lr.getMaxIter())

    # Create pipeline
    pipeline_lr_hpo = Pipeline(stages=[mmScaler, lr])
    
    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_lr_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, elasticNetParam=0.0)
print(f'The trained logistic regression achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
ElasticNet: 0.0
maxIter: 100


2022/09/13 04:19:43 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


The trained logistic regression achieved an AUROC score of 0.9870101538778137 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  elasticNetParam = params['elasticNetParam']

  # Train model
  model, f1_score = train_model(regParam, elasticNetParam)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
  
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'elasticNetParam': hp.quniform('elasticNetParam', 0, 1, q=0.01),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=tpe.suggest,
      max_evals=100
  )


Trial Parameters:
regParam:
0.86
ElasticNet:
0.63
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.04
ElasticNet:
0.84
maxIter:
100
Trial loss:
0.050097109018740205

Trial Parameters:
regParam:
0.33
ElasticNet:
0.06
maxIter:
100
Trial loss:
0.07452480413346052

Trial Parameters:
regParam:
0.92
ElasticNet:
0.63
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.47000000000000003
ElasticNet:
0.99
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.73
ElasticNet:
0.51
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.59
ElasticNet:
0.9400000000000001
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.44
ElasticNet:
0.66
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.9
ElasticNet:
0.18
maxIter:
100
Trial loss:
0.13742002016768629

Trial Parameters:
regParam:
0.08
ElasticNet:
0.18
maxIter:
100
Trial loss:
0.048690070352408465

Trial Parameters:
regParam:
0.65
ElasticNet:
0.78
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regPara

In [ ]:
# Print best params
best_params

{'elasticNetParam': 0.5, 'regParam': 0.0}

In [ ]:
# Define vars to fit best params
best_elasticNetParam = best_params['elasticNetParam']
best_regParam = best_params['regParam']

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_elasticNetParam, best_regParam) 
print(f'The trained logistic regression achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
regParam: 0.5
ElasticNet: 0.0
maxIter: 100
The trained logistic regression achieved an AUROC score of 0.902863788967094 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9860501025753575, and the final (tuned) model achieved 0.902863788967094.


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9864204599207805, and the final (tuned) model achieved 0.841991995620042.


In [ ]:
# Set up logistic regression pipeline
lr = LogisticRegression(labelCol='label', 
                        featuresCol='scaledFeatures', 
                        family='binomial', 
                        maxIter=100, 
                        regParam=0.0, 
                        elasticNetParam=0.0)

pipeline_lr = Pipeline(stages=[mmScaler, lr])
search_time_start = time.time()
pipelineModel_lr_hyperopt = pipeline_lr.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lr_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lr_hyperopt_us_auroc_100trials')

2022/10/08 19:14:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c334afa9a5714dad8d65d4dd508bacfd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/08 19:19:10 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit best model: 314.65425634384155


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hyperopt.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('Hyperopt: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

Hyperopt: Logistic Regression
Area under ROC curve: 0.98701
Test Error: 0.0129898 
Accuracy: 0.9858303292922332


### F1 Score 

#### 100 Trials

In [ ]:
def train_model(regParam, elasticNetParam): 
  
  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # MinMaxScaler
    mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                            outputCol='scaledFeatures',
                            min=0, max=1)
    
    # Define model
    lr = LogisticRegression(labelCol='label', 
                            featuresCol='unscaledFeatures',
                            featuresCol='scaledFeatures',
                            family='binomial',                             
                            regParam=regParam,
                            elasticNetParam=elasticNetParam,
                            standardization=True)
    
    print('\nTrial Parameters:')
    print('regParam:', lr.getRegParam())
    print('ElasticNet:', lr.getElasticNetParam())
    print('maxIter:', lr.getMaxIter())

    # Create pipeline
    pipeline_lr_hpo = Pipeline(stages=[mmScaler, lr])
    
    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_lr_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, elasticNetParam=0.0)
print(f'The trained logistic regression achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
ElasticNet: 0.0
maxIter: 100


2022/09/09 15:59:36 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


The trained logistic regression achieved an F1 score of 0.985674607837487 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  elasticNetParam = params['elasticNetParam']

  # Train model
  model, f1_score = train_model(regParam, elasticNetParam)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
  
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam', 0, 1, q=0.01),
  'elasticNetParam': hp.quniform('elasticNetParam', 0, 1, q=0.01),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=tpe.suggest,
      max_evals=100
  )


Trial Parameters:
regParam:
0.84
ElasticNet:
0.23
maxIter:
100
Trial loss:
0.0789872051321382

Trial Parameters:
regParam:
0.97
ElasticNet:
0.88
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.79
ElasticNet:
0.41000000000000003
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.87
ElasticNet:
0.43
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.8300000000000001
ElasticNet:
0.97
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.74
ElasticNet:
0.36
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.93
ElasticNet:
0.53
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.01
ElasticNet:
0.62
maxIter:
100
Trial loss:
0.03022188722092911

Trial Parameters:
regParam:
0.15
ElasticNet:
0.43
maxIter:
100
Trial loss:
0.13388752295843553

Trial Parameters:
regParam:
0.02
ElasticNet:
0.31
maxIter:
100
Trial loss:
0.04717663315753495

Trial Parameters:

In [ ]:
# Print best params
best_params

{'elasticNetParam': 0.01, 'regParam': 0.0}

In [ ]:
# Define vars to fit best params
best_elasticNetParam = best_params['elasticNetParam']
best_regParam = best_params['regParam']

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_elasticNetParam, best_regParam) 
print(f'The trained logistic regression achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
regParam: 0.01
ElasticNet: 0.0
maxIter: 100
The trained logistic regression achieved an F1 score of 0.9595326314111645 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.985674607837487, and the final (tuned) model achieved 0.9595326314111645.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9870101538778137, and the final (tuned) model achieved 0.9692043825742389.


In [ ]:
# Set up logistic regression pipeline
lr = LogisticRegression(labelCol='label', 
                        featuresCol='scaledFeatures', 
                        family='binomial', 
                        maxIter=100, 
                        regParam=0.0, 
                        elasticNetParam=0.0)

pipeline_lr = Pipeline(stages=[mmScaler, lr])
search_time_start = time.time()
pipelineModel_lr_hyperopt = pipeline_lr.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lr_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lr_hyperopt_us_f1_100trials')

2022/10/08 19:21:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'da298dccbf224446971f51376b98a330', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 230.4148120880127


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hyperopt.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('Hyperopt: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

Hyperopt: Logistic Regression
Area under ROC curve: 0.98701
Test Error: 0.0129898 
Accuracy: 0.9858303292922332


## LinearSVC

### AUROC Score 

#### 100 Trials

In [ ]:
def train_model(regParam, tol, maxIter):
  
  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Standard scaler
    stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                               outputCol='scaledFeatures', 
                               withStd=True, 
                               withMean=False)
    
    # Define model
    lsvc = LinearSVC(labelCol='label', 
                     featuresCol='scaledFeatures',
                     regParam=regParam,
                     tol=tol,
                     maxIter=maxIter)

    print('\nTrial Parameters:')
    print('regParam:', lsvc.getRegParam())
    print('tol:', lsvc.getTol())
    print('maxIter:', lsvc.getMaxIter())

    # Create pipeline
    pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, lsvc])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_lsvc_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, tol=1e-5, maxIter=100)
print(f'The trained LinearSVC achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 1e-05
maxIter: 100


2022/09/11 15:28:24 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


The trained LinearSVC achieved an AUROC score of 0.9788742585676203 on the validation data


In [ ]:
def train_with_hyperopt(params):
  
  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  tol = params['tol']
  maxIter = int(params['maxIter'])

  # Train model
  model, AUROC_score = train_model(regParam, tol, maxIter)

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'tol': hp.quniform('tol', 0, 1e-3, q=5e-8),
  'maxIter': scope.int(hp.quniform('maxIter', 100, 1000, q=100)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=100
  )

                                                       
Trial Parameters:
regParam:                                              
0.27                                                   
tol:                                                   
0.000638                                               
maxIter:                                               
400                                                    
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                            
0.06659555353993885                                    
                                                                                    
Trial Parameters:
regParam:                                                                           
0.76                                                                                
tol:                                                                                
0.0009021999999999999                                                               
maxIter:                                                                            
900                                                                                 
  1%|          | 1/100 [01:19<2:09:51, 78.70s/trial, best loss: 0.06659555353993885]

Trial loss:                                                                         
0.10439184129203471                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.25                                                                                
tol:                                                                                
0.0001759                                                                           
maxIter:                                                                            
800                                                                                 
  2%|▏         | 2/100 [02:37<2:08:31, 78.68s/trial, best loss: 0.06659555353993885]

Trial loss:                                                                         
0.06430090346162265                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.88                                                                                
tol:                                                                                
0.0004911                                                                           
maxIter:                                                                            
200                                                                                 
  3%|▎         | 3/100 [04:24<2:27:59, 91.54s/trial, best loss: 0.06430090346162265]

Trial loss:                                                                         
0.11008762103410463                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.31                                                                                
tol:                                                                                
9.515e-05                                                                           
maxIter:                                                                            
100                                                                                 
  4%|▍         | 4/100 [05:43<2:18:36, 86.63s/trial, best loss: 0.06430090346162265]

...

Trial loss:                                                                          
0.05155082188437787                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.24                                                                                 
tol:                                                                                 
0.0009069                                                                            
maxIter:                                                                             
500                                                                                  
 96%|█████████▌| 96/100 [2:36:09<06:02, 90.51s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.06313753535801336                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.9400000000000001                                                                   
tol:                                                                                 
0.0009406999999999999                                                                
maxIter:                                                                             
400                                                                                  
 97%|█████████▋| 97/100 [2:37:28<04:21, 87.18s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.11262167172716175                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.1                                                                                  
tol:                                                                                 
0.00035184999999999996                                                               
maxIter:                                                                             
200                                                                                  
 98%|█████████▊| 98/100 [2:38:46<02:48, 84.26s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.04420100533688376                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.06                                                                                 
tol:                                                                                 
0.0007159499999999999                                                                
maxIter:                                                                             
300                                                                                  
 99%|█████████▉| 99/100 [2:40:13<01:25, 85.15s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.037522653304294384                                                                 
100%|██████████| 100/100 [2:41:40<00:00, 97.01s/trial, best loss: 0.01981533806748803]


In [ ]:
# Print best params
best_params

{'maxIter': 500.0, 'regParam': 0.0, 'tol': 0.0009712999999999999}

In [ ]:
# Define vars to fit best params
best_tol = best_params['tol']
best_regParam = best_params['regParam']
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_auroc_score = train_model(best_regParam, best_tol, best_maxIter)
print(f'The trained LinearSVC achieved an AUROC score of {val_auroc_score} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 0.0009712999999999999
maxIter: 500


The trained LinearSVC achieved an AUROC score of 0.980184661932512 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9788742585676203, and the final (tuned) model achieved 0.980184661932512.


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9797006472779239, and the final (tuned) model achieved 0.9809885079539138.


In [ ]:
# Set up LinearSVC pipeline: {'maxIter': 500.0, 'regParam': 0.0, 'tol': 0.0009712999999999999}
lsvc = LinearSVC(labelCol='label', 
                 featuresCol='scaledFeatures', 
                 maxIter=best_maxIter, 
                 regParam=best_regParam, 
                 tol=best_tol)

pipeline_lsvc = Pipeline(stages=[stdScaler, lsvc])
search_time_start = time.time()
pipelineModel_lsvc_hyperopt = pipeline_lsvc.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lsvc_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lsvc_hyperopt_us_auroc')

2022/09/15 04:31:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7f8346f0edd2487c8fc18ed703b9d4d7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/09/15 04:38:39 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit best model: 437.71806812286377


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hyperopt.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('Hyperopt: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lsvc)) 

Hyperopt: LinearSVC
Area under ROC curve: 0.980185
Test Error: 0.0198153
Accuracy: 0.9812543210790038


### F1 Score

#### 100 Trials

In [ ]:
def train_model(regParam, tol, maxIter):
  
  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Standard scaler
    stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                               outputCol='scaledFeatures', 
                               withStd=True, 
                               withMean=False)
    
    # Define model
    lsvc = LinearSVC(labelCol='label', 
                     featuresCol='scaledFeatures',
                     regParam=regParam,
                     tol=tol,
                     maxIter=maxIter)

    print('\nTrial Parameters:')
    print('regParam:', lsvc.getRegParam())
    print('tol:', lsvc.getTol())
    print('maxIter:', lsvc.getMaxIter())

    # Create pipeline
    pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, lsvc])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_lsvc_hpo.fit(trainDF)   

    # Evaluate the model on the validation set
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, tol=1e-5, maxIter=100)
print(f'The trained LinearSVC achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 1e-05
maxIter: 100


The trained LinearSVC achieved an F1 score of 0.9797006472779239 on the validation data


In [ ]:
def train_with_hyperopt(params):
  
  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  tol = params['tol']
  maxIter = int(params['maxIter'])

  # Train model
  model, f1_score = train_model(regParam, tol, maxIter)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam', 0, 1, q=0.01),
  'tol': hp.quniform('tol', 0, 1e-3, q=5e-8),
  'maxIter': scope.int(hp.quniform('maxIter', 100, 1000, q=100)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=100
  )

                                                       
Trial Parameters:
regParam:                                              
0.19                                                   
tol:                                                   
0.00029985                                             
maxIter:                                               
600                                                    
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                            
0.09194180341295588                                    
                                                                                    
Trial Parameters:
regParam:                                                                           
1.0                                                                                 
tol:                                                                                
0.0005193                                                                           
maxIter:                                                                            
1000                                                                                
  1%|          | 1/100 [01:32<2:32:03, 92.16s/trial, best loss: 0.09194180341295588]

Trial loss:                                                                         
0.1747979565231944                                                                  
                                                                                    
Trial Parameters:
regParam:                                                                           
0.97                                                                                
tol:                                                                                
4.965e-05                                                                           
maxIter:                                                                            
400                                                                                 
  2%|▏         | 2/100 [02:45<2:12:14, 80.96s/trial, best loss: 0.09194180341295588]

Trial loss:                                                                         
0.17334211412520195                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.56                                                                                
tol:                                                                                
0.00074595                                                                          
maxIter:                                                                            
900                                                                                 
  3%|▎         | 3/100 [04:04<2:09:10, 79.90s/trial, best loss: 0.09194180341295588]

Trial loss:                                                                         
0.14863572908268674                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.32                                                                                
tol:                                                                                
0.0001602                                                                           
maxIter:                                                                            
700                                                                                 
  4%|▍         | 4/100 [05:17<2:03:17, 77.05s/trial, best loss: 0.09194180341295588]

...

Trial loss:                                                                          
0.10457512810656122                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.06                                                                                 
tol:                                                                                 
0.0007941                                                                            
maxIter:                                                                             
300                                                                                  
 96%|█████████▌| 96/100 [2:16:40<05:23, 80.76s/trial, best loss: 0.01877511196532078]

Trial loss:                                                                          
0.04118632022545532                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.12                                                                                 
tol:                                                                                 
0.0009996                                                                            
maxIter:                                                                             
700                                                                                  
 97%|█████████▋| 97/100 [2:18:01<04:02, 80.92s/trial, best loss: 0.01877511196532078]

Trial loss:                                                                          
0.06831976238019155                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.08                                                                                 
tol:                                                                                 
0.0008319499999999999                                                                
maxIter:                                                                             
300                                                                                  
 98%|█████████▊| 98/100 [2:19:20<02:40, 80.27s/trial, best loss: 0.01877511196532078]

Trial loss:                                                                          
0.05101619231700982                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.68                                                                                 
tol:                                                                                 
0.00052775                                                                           
maxIter:                                                                             
400                                                                                  
 99%|█████████▉| 99/100 [2:20:41<01:20, 80.40s/trial, best loss: 0.01877511196532078]

Trial loss:                                                                          
0.1580426901025087                                                                   
100%|██████████| 100/100 [2:21:54<00:00, 85.14s/trial, best loss: 0.01877511196532078]


In [ ]:
# Print best params
best_params

{'maxIter': 300.0, 'regParam': 0.0, 'tol': 0.00078725}

In [ ]:
# Define vars to fit best params
best_tol = best_params['tol']
best_regParam = best_params['regParam']
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_regParam, best_tol, best_maxIter)
print(f'The trained LinearSVC achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 0.00078725
maxIter: 300


The trained LinearSVC achieved an F1 score of 0.9812248880346792 on the validation data


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9797006472779239, and the final (tuned) model achieved 0.9812248880346792.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9788742585676203, and the final (tuned) model achieved 0.9801636688553507.


In [ ]:
# Set up LinearSVC pipeline: {'maxIter': 300.0, 'regParam': 0.0, 'tol': 0.00078725}
lsvc = LinearSVC(labelCol='label',
                 featuresCol='scaledFeatures',
                 regParam=best_regParam,
                 tol=best_tol,
                 maxIter=best_maxIter)

pipeline_lsvc = Pipeline(stages=[stdScaler, lsvc])
search_time_start = time.time()
pipelineModel_lsvc_hyperopt = pipeline_lsvc.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lsvc_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lsvc_hyperopt_us_f1')

2022/09/15 04:39:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c3a1e2c96e214a278811f1e1aa148bb6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 479.29352951049805


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hyperopt.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('Hyperopt: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lsvc)) 

Hyperopt: LinearSVC
Area under ROC curve: 0.980164
Test Error: 0.0198363
Accuracy: 0.9814716756884245


## Decision Tree

### AUROC Score

##### 50 Trials 

In [ ]:
def train_model(maxDepth, maxBins):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    dtc = DecisionTreeClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', dtc.getMaxDepth())
    print('maxBins:', dtc.getMaxBins())
    
    # Create pipeline
    pipeline_dt_hpo = Pipeline(stages=[dtc])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_dt_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=16)
print('Time to fit model:', time.time() - search_time_start)
print(f'The trained decision tree classifier achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 16


Time to fit model: 98.48372387886047
The trained decision tree classifier achieved an AUROC score of 0.9132889853108634 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins)
  
  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)
    
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 30, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 128, q=16)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin(
    fn=train_with_hyperopt,
    space=space,
    algo=algo,
    max_evals=50
  )

                                                      
Trial Parameters:
maxDepth:                                             
29                                                    
maxBins:                                              
48                                                    
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2022/09/14 20:35:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpun6si8ci/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                           
0.04159002702380965                                   
                                                                                    
Trial Parameters:
maxDepth:                                                                           
23                                                                                  
maxBins:                                                                            
16                                                                                  
  2%|▏         | 1/50 [02:37<2:08:20, 157.15s/trial, best loss: 0.04159002702380965]

2022/09/14 20:37:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpvng1p8my/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                         
0.04732732857792721                                                                 
                                                                                    
Trial Parameters:
maxDepth:                                                                           
19                                                                                  
maxBins:                                                                            
96                                                                                  
  4%|▍         | 2/50 [04:39<1:47:58, 134.97s/trial, best loss: 0.04159002702380965]

...

Trial loss:                                                                           
0.03889618742722534                                                                   
                                                                                      
Trial Parameters:
maxDepth:                                                                             
17                                                                                    
maxBins:                                                                              
32                                                                                    
 94%|█████████▍| 47/50 [1:38:39<06:42, 134.12s/trial, best loss: 0.037492525901350326]

Trial loss:                                                                           
0.044008478364794934                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
25                                                                                    
maxBins:                                                                              
48                                                                                    
 96%|█████████▌| 48/50 [1:40:07<04:00, 120.19s/trial, best loss: 0.037492525901350326]

2022/09/14 22:14:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpraqgmydb/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.039392081941877155                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
15                                                                                    
maxBins:                                                                              
64                                                                                    
 98%|█████████▊| 49/50 [1:42:29<02:06, 126.85s/trial, best loss: 0.037492525901350326]

Trial loss:                                                                           
0.042318302537308394                                                                  
100%|██████████| 50/50 [1:43:50<00:00, 124.61s/trial, best loss: 0.037492525901350326]


In [ ]:
# Print best params
best_params

{'maxBins': 80.0, 'maxDepth': 26.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins)
print(f'The trained decision tree classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 26
maxBins: 80


2022/09/14 22:18:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8ni_1d8d/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.


The trained decision tree classifier achieved an AUROC score of 0.9625074740986497 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9132889853108634, and the final (tuned) model achieved 0.9625074740986497.


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9181702970821815, and the final (tuned) model achieved 0.9747492554530752.


In [ ]:
# Set up DecisionTree pipeline - maxDepth=26, maxBins=80
dt = DecisionTreeClassifier(labelCol='label', 
                            featuresCol='unscaledFeatures', 
                            impurity='gini', 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,                            
                            seed=seed_value)

search_time_start = time.time()
pipeline_dt = Pipeline(stages=[dt])
pipelineModel_dt_hyperopt = pipeline_dt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_dt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_dt_hyperopt_us_auroc')

2022/09/15 11:02:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd9e467bb76ca4457b0aa4c9869250d33', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/09/15 11:05:57 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/09/15 11:06:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp463qz_oh/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 282.30774688720703


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hyperopt.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('Hyperopt: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

Hyperopt: Decision Tree
Area under ROC curve: 0.962507
Test Error: 0.0374925 
Accuracy: 0.974555636999304


### F1 Score

##### 50 Trials 

In [ ]:
def train_model(maxDepth, maxBins):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    dtc = DecisionTreeClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', dtc.getMaxDepth())
    print('maxBins:', dtc.getMaxBins())
    
    # Create pipeline
    pipeline_dt_hpo = Pipeline(stages=[dtc])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')
    # Fit model
    model = pipeline_dt_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=16)
print('Time to fit model:', time.time() - search_time_start)
print(f'The trained decision tree classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 16


2022/09/14 22:44:15 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


Time to fit model: 108.95203232765198
The trained decision tree classifier achieved an F1 score of 0.9181702970821815 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins)
  
  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
    
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 30, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 128, q=16)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin(
    fn=train_with_hyperopt,
    space=space,
    algo=algo,
    max_evals=50
  )

                                                      
Trial Parameters:
maxDepth:                                             
25                                                    
maxBins:                                              
48                                                    
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2022/09/14 22:46:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppk024urn/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                           
0.025807941057547024                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
19                                                                                   
maxBins:                                                                             
64                                                                                   
  2%|▏         | 1/50 [02:10<1:46:21, 130.23s/trial, best loss: 0.025807941057547024]

Trial loss:                                                                          
0.02168921980563765                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                           
20                                                                                  
maxBins:                                                                            
48                                                                                  
  4%|▍         | 2/50 [03:54<1:25:29, 106.86s/trial, best loss: 0.02168921980563765]

Trial loss:                                                                         
0.02319272314376941                                                                 
                                                                                    
Trial Parameters:
maxDepth:                                                                           
15                                                                                  
maxBins:                                                                            
64                                                                                  
  6%|▌         | 3/50 [05:41<1:24:15, 107.56s/trial, best loss: 0.02168921980563765]

...

Trial loss:                                                                           
0.022594621285158212                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
15                                                                                    
maxBins:                                                                              
112                                                                                   
 92%|█████████▏| 46/50 [1:12:31<06:59, 104.91s/trial, best loss: 0.019099949448309173]

Trial loss:                                                                           
0.019099949448309173                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                            
15                                                                                   
maxBins:                                                                             
112                                                                                  
 94%|█████████▍| 47/50 [1:14:07<04:54, 98.02s/trial, best loss: 0.019099949448309173]

Trial loss:                                                                          
0.019099949448309173                                                                 
                                                                                     
Trial Parameters:
maxDepth:                                                                            
21                                                                                   
maxBins:                                                                             
128                                                                                  
 96%|█████████▌| 48/50 [1:15:37<03:14, 97.19s/trial, best loss: 0.019099949448309173]

Trial loss:                                                                          
0.022345093722938802                                                                 
                                                                                      
Trial Parameters:
maxDepth:                                                                             
19                                                                                    
maxBins:                                                                              
112                                                                                   
 98%|█████████▊| 49/50 [1:17:36<01:42, 102.89s/trial, best loss: 0.019099949448309173]

Trial loss:                                                                           
0.021322080112109676                                                                  
100%|██████████| 50/50 [1:19:08<00:00, 94.97s/trial, best loss: 0.019099949448309173] 


In [ ]:
# Print best params
best_params

{'maxBins': 112.0, 'maxDepth': 15.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins)
print(f'The trained decision tree classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 15
maxBins: 112


The trained decision tree classifier achieved an F1 score of 0.9809000505516908 on the validation data


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9181702970821815, and the final (tuned) model achieved 0.9809000505516908.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9132889853108634, and the final (tuned) model achieved 0.9564704240277363.


In [ ]:
# Set up decision tree pipeline: {'maxBins': 112.0, 'maxDepth': 15.0}
dt = DecisionTreeClassifier(labelCol='label', 
                            featuresCol='unscaledFeatures', 
                            impurity='gini', 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_dt = Pipeline(stages=[dt])
pipelineModel_dt_hyperopt = pipeline_dt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_dt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_dt_hyperopt_us_f1')

2022/09/17 17:53:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0db3305a8a4f490cb0f5da9739815a70', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 119.21714472770691


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hyperopt.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('Hyperopt: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

Hyperopt: Decision Tree
Area under ROC curve: 0.95647
Test Error: 0.0435296 
Accuracy: 0.9809121031833201


## Random Forest

### AUROC Score

#### 30 Trials 

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', rf.getMaxDepth())
    print('maxBins:', rf.getMaxBins())	    
    print('numTrees:', rf.getNumTrees())

    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print(f'The trained random forest classifier achieved an AUROC score of {val_metric} on the validation data')

2022/09/13 19:44:25 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


The trained random forest classifier achieved an AUROC score of 0.9604714793841286 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins, numTrees) 

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss: ', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 10, 30, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )


Trial Parameters:
maxDepth:
7
maxBins:
20
numTrees:
12

Trial Loss:
0.03130775047532797

Trial Parameters:
maxDepth:
6
maxBins:
16
numTrees:
29

Trial Loss:
0.03130775047532797

Trial Parameters:
maxDepth:
8
maxBins:
36
numTrees:
30

Trial Loss:
0.0287370391745686

Trial Parameters:
maxDepth:
11
maxBins:
20
numTrees:
23
  10%|█         | 3/30 [12:43<2:03:53, 275.30s/it, best loss: 0.0287370391745686] 2022/09/13 23:19:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp6yycu5ik/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.022400250543279432

Trial Parameters:
maxDepth:
18
maxBins:
24
numTrees:
25
 13%|█▎        | 4/30 [20:54<2:36:14, 360.57s/it, best loss: 0.022400250543279432] 2022/09/13 23:47:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpkgidgx4k/model, f

In [ ]:
# Print best params
best_params

{'maxBins': 44.0, 'maxDepth': 20.0, 'numTrees': 21.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params and evaluate AUROC on tuned model
final_model, val_auroc_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained random forest classifier achieved an AUROC score of {val_auroc_score} on the validation data')


Trial Parameters:
maxDepth: 20
maxBins: 44
numTrees: 21
2022/09/14 06:04:17 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpxqy21gif/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained random forest classifier achieved an AUROC score of 0.9838385238343885 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9604714793841286, and the final (tuned) model achieved 0.9838385238343885.


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9218991286344467, and the final (tuned) model achieved 0.9840041624168372.


In [ ]:
# Set up RF pipeline: {'maxBins': 44.0, 'maxDepth': 20.0, 'numTrees': 21.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_us_auroc_30trials')

2022/10/09 01:20:06 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '602336240f8d4d5fa2feac24305b0d1d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 01:57:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp4phmqx1x/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 2259.5389306545258


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.984546
Test Error: 0.015454 
Accuracy: 0.9845169525033933


#### 20 Trials - Increased Trees & Depth

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)
    
    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    print('\nTrial Parameters:')
    print('maxDepth:', rf.getMaxDepth())
    print('maxBins:', rf.getMaxBins())	    
    print('numTrees:', rf.getNumTrees())
    
    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained Random Forest classifier achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 32
numTrees: 10


2022/09/16 01:52:44 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit initial model: 193.86712265014648
The trained Random Forest classifier achieved an AUROC score of 0.9604714793841286 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, numTrees)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 10, 30, q=5)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 32, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 20, 50, q=5)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=20
  )


Trial Parameters:
maxDepth:
20
maxBins:
28
numTrees:
30
  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?] 2022/09/16 02:32:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpi_xgdr8s/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.016022073724315522

Trial Parameters:
maxDepth:
10
maxBins:
24
numTrees:
20
  5%|▌         | 1/20 [39:37<12:32:47, 2377.24s/it, best loss: 0.016022073724315522] 2022/09/16 02:38:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ymx_q_x/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.025062577752743675

Trial Parameters:
maxDepth:
20
maxBins:
20
numTrees:
40
 10%|█         | 2/20 [45:20<5:54:13, 1180.76s/it, best loss: 0.016022073724315522] 2022/09/1

In [ ]:
# Print best params
best_params

{'maxBins': 32.0, 'maxDepth': 20.0, 'numTrees': 50.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params and evaluate F1 tuned model
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins, 
                                           best_numTrees)
print(f'The trained Random Forest classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 20
maxBins: 32
numTrees: 50
2022/09/17 03:43:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpbqohe6b7/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an AUROC score of 0.9847620883910223 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9218991286344467, and the final (tuned) model achieved 0.9845615476056315.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9604714793841286, and the final (tuned) model achieved 0.9847620883910223.


In [ ]:
# Set up RF pipeline: {'maxBins': 32.0, 'maxDepth': 20.0, 'numTrees': 50.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_us_auroc_moreParams')

2022/10/09 02:25:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '480c448951584f04be599d418d84f3f1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 04:09:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprkyvoz8a/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 6244.508635759354


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.984949
Test Error: 0.015051 
Accuracy: 0.9847504930943666


### F1 Score

#### 30 Trials - Stopped

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)
    
    print('\nTrial Parameters:')
    print('maxDepth:', rf.getMaxDepth())
    print('maxBins:', rf.getMaxBins())	    
    print('numTrees:', rf.getNumTrees())

    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained Random Forest classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 32
numTrees: 10


2022/09/14 13:14:03 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit initial model: 203.0401062965393
The trained Random Forest classifier achieved an F1 score of 0.9218991286344467 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, numTrees)

  # Return a loss to monitor
  loss = 1 -f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 10, 30, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )


Trial Parameters:
maxDepth:
6
maxBins:
48
numTrees:
11

Trial Loss:
0.06143100971349025

Trial Parameters:
maxDepth:
7
maxBins:
8
numTrees:
23

Trial Loss:
0.06830745465679477

Trial Parameters:
maxDepth:
10
maxBins:
60
numTrees:
20
  7%|▋         | 2/30 [06:17<1:31:22, 195.81s/it, best loss: 0.06143100971349025] 2022/09/14 13:26:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpy7vdkg8p/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.04437637962553198

Trial Parameters:
maxDepth:
19
maxBins:
8
numTrees:
19
 10%|█         | 3/30 [12:09<2:00:07, 266.95s/it, best loss: 0.04437637962553198] 2022/09/14 13:48:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp0kwm7u42/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the

In [ ]:
# Print best params
best_params

{'maxBins': 40.0, 'maxDepth': 20.0, 'numTrees': 28.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params and evaluate F1 on tuned model
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained Random Forest classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 20
maxBins: 40
numTrees: 28
2022/09/15 00:00:47 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpl4hgol07/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an F1 score of 0.9846789998334257 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9218991286344467, and the final (tuned) model achieved 0.9846789998334257.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9604714793841286, and the final (tuned) model achieved 0.9845886468846079


In [ ]:
# Set up RF pipeline: {'maxBins': 40.0, 'maxDepth': 20.0, 'numTrees': 28.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

earch_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_us_f1_30trials')

2022/10/09 04:11:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1137130c851b4bd3a0495e9f138a9f39', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 05:03:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpm0jwngh2/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 9479.31393790245


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.984589
Test Error: 0.0154114 
Accuracy: 0.9849354757406821


#### 20 Trials - Increased Trees & Depth

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)
    
    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained Random Forest classifier achieved an F1 score of {val_metric} on the validation data')

2022/09/11 15:16:10 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit initial model: 202.04475450515747
The trained Random Forest classifier achieved an F1 score of 0.9218991286344467 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, numTrees)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 10, 30, q=5)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 32, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 20, 50, q=5)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=20
  )

  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?] 2022/09/11 15:34:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpwviovdqo/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.023243266161836362
  5%|▌         | 1/20 [18:27<5:50:42, 1107.52s/it, best loss: 0.023243266161836362] 2022/09/11 16:24:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpe2ro588y/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.015016700738360589
 10%|█         | 2/20 [1:08:40<11:08:30, 2228.39s/it, best loss: 0.015016700738360589] 2022/09/11 17:28:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp3h6rvtk0/model, flavor: spark), fall ba

In [ ]:
# Print best params
best_params

{'maxBins': 32.0, 'maxDepth': 30.0, 'numTrees': 35.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params and evaluate F1 tuned model
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, 
                                        best_numTrees)
print(f'The trained Random Forest classifier achieved an F1 score of {val_f1_score} on the validation data')

2022/09/12 10:39:57 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprqdytjaz/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an F1 score of 0.9855214900118682 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9218991286344467, and the final (tuned) model achieved 0.9855214900118682.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9604714793841286, and the final (tuned) model achieved 0.9828512581694128.


In [ ]:
# Set up RF pipeline: {'maxBins': 32.0, 'maxDepth': 30.0, 'numTrees': 35.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_us_f1_moreParams')

2022/10/09 19:35:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9f1674ed7ea846e7a63f5a0c40aae6cd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 21:35:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpmoc9o8iu/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 7220.566922426224


In [ ]:
# Predict on test set
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

# AUROC and Test Error
rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.982851
Test Error: 0.0171487 
Accuracy: 0.9858557644061017


## Gradient Boosted Trees

### AUROC Score

#### 30 Trials

In [ ]:
def train_model(maxDepth, maxBins, maxIter):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    gbt = GBTClassifier(labelCol='label',
                       featuresCol='unscaledFeatures',
                       maxDepth=maxDepth,
                       maxBins=maxBins,
                       maxIter=maxIter,
                       seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', gbt.getMaxDepth())
    print('maxBins:', gbt.getMaxBins())
    print('maxIter:', gbt.getMaxIter())
    
    # Create pipeline
    pipeline_gbt_hpo = Pipeline(stages=[gbt])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_gbt_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, maxIter=10)
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained gradient booster tree classifier achieved an AUROC score of {val_metric} on the validation data')

Trial Parameters:
maxDepth: 5
maxBins: 32
maxIter: 10
2022/09/14 01:12:48 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.
Time to fit initial model: 169.96657633781433
The trained gradient boosted tree classifier achieved an AUROC score of 0.9727056090793263 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  maxIter = int(params['maxIter'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins, maxIter)

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'maxIter': scope.int(hp.quniform('maxIter', 5, 20, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )

                                                      
Trial Parameters:
maxDepth:                                             
18                                                    
maxBins:                                              
24                                                    
maxIter:                                              
9                                                     
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2022/09/14 01:22:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpgkcqyox_/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                           
0.021595029023087853                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
8                                                                                    
maxBins:                                                                             
64                                                                                   
maxIter:                                                                             
15                                                                                   
  3%|▎         | 1/30 [09:08<4:24:43, 547.71s/trial, best loss: 0.021595029023087853]

Trial loss:                                                                          
0.015241522907011418                                                                 
                                                                                     
Trial Parameters:
maxDepth:                                                                            
7                                                                                    
maxBins:                                                                             
8                                                                                    
maxIter:                                                                             
9                                                                                    
  7%|▋         | 2/30 [13:22<2:55:04, 375.14s/trial, best loss: 0.015241522907011418]

Trial loss:                                                                          
0.01938082626859472                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
16                                                                                   
maxBins:                                                                             
24                                                                                   
maxIter:                                                                             
10                                                                                   
 10%|█         | 3/30 [15:57<2:03:33, 274.56s/trial, best loss: 0.015241522907011418]

...

Trial loss:                                                                           
0.013378619552655624                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
11                                                                                    
maxBins:                                                                              
20                                                                                    
maxIter:                                                                              
16                                                                                    
 93%|█████████▎| 28/30 [3:17:22<13:09, 394.92s/trial, best loss: 0.013322603686918177]

2022/09/14 04:36:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpwsjexb_p/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.013997665958153216                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
13                                                                                    
maxBins:                                                                              
24                                                                                    
maxIter:                                                                              
19                                                                                    
 97%|█████████▋| 29/30 [3:23:14<06:22, 382.01s/trial, best loss: 0.013322603686918177]

2022/09/14 04:45:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmperzztngr/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.014434734861480614                                                                  
100%|██████████| 30/30 [3:32:07<00:00, 424.25s/trial, best loss: 0.013322603686918177]


In [ ]:
# Print best params
best_params

{'maxBins': 28.0, 'maxDepth': 10.0, 'maxIter': 20.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins, best_maxIter)
print(f'The trained gradient boosted tree classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 10
maxBins: 28
numTrees: 20
2022/09/17 16:34:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpegojnmdx/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained gradient boosted tree classifier achieved an AUROC score of 0.9866776626412849 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9727056090793263, and the final (tuned) model achieved 0.9866773963130818.


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.testDF')

On the test data, the initial (untuned) model achieved F1 score 0.9456931214611761, and the final (tuned) model achieved 0.9810338130049403.


In [ ]:
# Set up GBT pipeline: {'maxBins': 28.0, 'maxDepth': 10.0, 'maxIter': 20.0}
gbt = GBTClassifier(labelCol='label', 
                    featuresCol='unscaledFeatures', 
                    maxDepth=best_maxDepth, 
                    maxBins=best_maxBins, 
                    maxIter=best_maxIter,
                    seed=seed_value)

search_time_start = time.time()
pipeline_gbt = Pipeline(stages=[gbt])
pipelineModel_gbt_hyperopt = pipeline_gbt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_gbt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_gbt_hyperopt_us_auroc')

2022/10/08 21:04:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '34322ad9bd5942f98f96f2bb83e31595', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/08 21:15:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp5wqtu8lz/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 621.5592732429504


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hyperopt.transform(testDF) 

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('Hyperopt: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

Hyperopt: Gradient Boosted Trees
Area under ROC curve: 0.986678
Test Error: 0.0133223 
Accuracy: 0.9810554647342146


### F1 Score

#### 30 Trials

In [ ]:
def train_model(maxDepth, maxBins, maxIter):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    gbt = GBTClassifier(labelCol='label', 
                        featuresCol='unscaledFeatures',
                        maxDepth=maxDepth,
                        maxBins=maxBins,
                        maxIter=maxIter,
                        seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', gbt.getMaxDepth())
    print('maxBins:', gbt.getMaxBins())
    print('maxIter:', gbt.getMaxIter())
    
    # Create pipeline
    pipeline_gbt_hpo = Pipeline(stages=[gbt])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')
    # Fit model
    model = pipeline_gbt_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, maxIter=10)
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained gradient booster tree classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 32
maxIter: 10


2022/09/15 16:39:06 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


Time to fit initial model: 164.79670524597168
The trained gradient booster tree classifier achieved an F1 score of 0.9456931214611761 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  maxIter = int(params['maxIter'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, maxIter)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'maxIter': scope.int(hp.quniform('maxIter', 5, 20, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )

                                                      
Trial Parameters:
maxDepth:                                             
10                                                    
maxBins:                                              
28                                                    
maxIter:                                              
17                                                    
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2022/09/15 16:44:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpdt9q35bu/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                           
0.019332690253043294                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
17                                                                                   
maxBins:                                                                             
28                                                                                   
maxIter:                                                                             
10                                                                                   
  3%|▎         | 1/30 [05:23<2:35:56, 322.65s/trial, best loss: 0.019332690253043294]

2022/09/15 16:53:55 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmponmkp7ff/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.019167905342976632                                                                 
                                                                                     
Trial Parameters:
maxDepth:                                                                            
17                                                                                   
maxBins:                                                                             
52                                                                                   
maxIter:                                                                             
16                                                                                   
  7%|▋         | 2/30 [14:36<3:33:47, 458.14s/trial, best loss: 0.019167905342976632]

...

Trial loss:                                                                           
0.017191591982601895                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
18                                                                                    
maxBins:                                                                              
32                                                                                    
maxIter:                                                                              
18                                                                                    
 93%|█████████▎| 28/30 [3:53:02<20:18, 609.10s/trial, best loss: 0.016807621171831322]

2022/09/15 20:55:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp0_3nz0nc/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.01888588363932764                                                                   
                                                                                      
Trial Parameters:
maxDepth:                                                                             
16                                                                                    
maxBins:                                                                              
48                                                                                    
maxIter:                                                                              
20                                                                                    
 97%|█████████▋| 29/30 [4:15:49<13:55, 835.58s/trial, best loss: 0.016807621171831322]

2022/09/15 21:13:17 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpi6u1ym0e/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.017441633532479295                                                                  
100%|██████████| 30/30 [4:34:01<00:00, 548.04s/trial, best loss: 0.016807621171831322]


In [ ]:
# Print best params
best_params

{'maxDepth': 15.0, 'maxBins': 44.0, 'maxIter': 20.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, 
                                        best_maxIter)
print(f'The trained gradient booster tree classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 15
maxBins: 44
maxIter: 20
2022/09/15 21:27:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpyv1djduo/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained gradient boosted tree classifier achieved an F1 score of 0.9831923788281687 on the validation data


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.testDF')

On the test data, the initial (untuned) model achieved F1 score 0.9456931214611761, and the final (tuned) model achieved 0.9831923788281687.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9727056090793263, and the final (tuned) model achieved 0.9848674053765734.


In [ ]:
# Set up GBT pipeline: {'maxDepth': 15.0, 'maxBins': 44.0, 'maxIter': 20.0}
gbt = GBTClassifier(labelCol='label', 
                    featuresCol='unscaledFeatures', 
                    maxDepth=best_maxDepth, 
                    maxBins=best_maxBins, 
                    maxIter=best_maxIter,
                    seed=seed_value)

search_time_start = time.time()
pipeline_gbt = Pipeline(stages=[gbt])
pipelineModel_gbt_hyperopt = pipeline_gbt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_gbt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_gbt_hyperopt_us_f1')

2022/10/10 00:04:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '28de140af29949e193318b26c0157cd7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/10 00:27:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzuzpr7s5/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 1325.9579446315765


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hyperopt.transform(testDF) 

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('Hyperopt: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

Hyperopt: Gradient Boosted Trees
Area under ROC curve: 0.984867
Test Error: 0.0151329 
Accuracy: 0.983296067037711


# SMOTE - Split Over Upsampling 

## Read Data and View Schema

In [ ]:
trainDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/trainDF_SMOTE.csv',
                         header=True, inferSchema=True)
trainDF.cache()
trainDF = trainDF \
  .withColumn('loan_amnt', trainDF['loan_amnt'].cast(IntegerType())) \
  .withColumn('revol_bal', trainDF['revol_bal'].cast(IntegerType())) \
  .withColumn('term_ 60 months', trainDF['term_ 60 months'].cast(IntegerType())) \
  .withColumn('grade_B', trainDF['grade_B'].cast(IntegerType())) \
  .withColumn('grade_C', trainDF['grade_C'].cast(IntegerType())) \
  .withColumn('grade_D', trainDF['grade_D'].cast(IntegerType())) \
  .withColumn('home_ownership_MORTGAGE', trainDF['home_ownership_MORTGAGE'].cast(IntegerType())) \
  .withColumn('home_ownership_OWN', trainDF['home_ownership_OWN'].cast(IntegerType())) \
  .withColumn('home_ownership_RENT', trainDF['home_ownership_RENT'].cast(IntegerType())) \
  .withColumn('verification_status_Source Verified', trainDF['verification_status_Source Verified'].cast(IntegerType())) \
  .withColumn('verification_status_Verified', trainDF['verification_status_Verified'].cast(IntegerType())) \
  .withColumn('purpose_credit_card', trainDF['purpose_credit_card'].cast(IntegerType())) \
  .withColumn('initial_list_status_w', trainDF['initial_list_status_w'].cast(IntegerType())) \
  .withColumn('application_type_Joint App', trainDF['application_type_Joint App'].cast(IntegerType())) \
  .withColumn('disbursement_method_DirectPay', trainDF['disbursement_method_DirectPay'].cast(IntegerType())) 
print('\nTrain Schema')
trainDF.printSchema()

testDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/testDF_SMOTE.csv',
                        header=True, inferSchema=True)
testDF.cache()
print('\nTest Schema')
testDF.printSchema()


Train Schema
root
 |-- loan_amnt: integer (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- collections_12_mths_ex_med: double (nullable = true)
 |-- acc_open_past_24mths: double (nullable = true)
 |-- bc_open_to_buy: double (nullable = true)
 |-- chargeoff_within_12_mths: double (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- mths_since_recent_bc: double (nullable = true)
 |-- num_bc_sats: double (nullable = true)
 |-- num_bc_tl: double (nullable = true)
 |-- num_sats: double (nulla

## Set up Vector Assembler

In [ ]:
# Define features and label for train data
features = trainDF.columns[0: len(trainDF.columns) - 1]
trainDF = trainDF.select(col('loan_status').alias('label'), *features)

# Transform train data
trainDF = vecAssembler.transform(trainDF)  

In [ ]:
# Define features and label for test data 
features = testDF.columns[0: len(testDF.columns) - 1]
testDF = testDF.select(col('loan_status').alias('label'), *features)

# Transform test data
testDF = vecAssembler.transform(testDF)  

In [ ]:
# Change directory to write mlruns
%cd /content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/

/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt


## Logistic Regression

### AUROC Score

#### 100 Trials

In [ ]:
def train_model(regParam, elasticNetParam): 
  
  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run
  with mlflow.start_run(nested=True):

    # MinMaxScaler
    mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                            outputCol='scaledFeatures',
                            min=0, max=1)
    
    # Define model
    lr = LogisticRegression(labelCol='label', 
                            featuresCol='scaledFeatures',
                            family='binomial',                             
                            regParam=regParam,
                            elasticNetParam=elasticNetParam,
                            standardization=True)
    
    print('\nTrial Parameters:')
    print('regParam:', lr.getRegParam())
    print('ElasticNet:', lr.getElasticNetParam())
    print('maxIter:', lr.getMaxIter())

    # Create pipeline
    pipeline_lr_hpo = Pipeline(stages=[mmScaler, lr])
    
    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_lr_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, elasticNetParam=0.0)
print(f'The trained logistic regression achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
ElasticNet: 0.0
maxIter: 100
The trained logistic regression achieved an AUROC score of 0.9860501025753575 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  elasticNetParam = params['elasticNetParam']

  # Train model
  model, f1_score = train_model(regParam, elasticNetParam)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
  
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'elasticNetParam': hp.quniform('elasticNetParam', 0, 1, q=0.01),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=tpe.suggest,
      max_evals=100
  )


Trial Parameters:
regParam:
0.5700000000000001
ElasticNet:
0.32
maxIter:
100
Trial loss:
0.1988924485125858

Trial Parameters:
regParam:
0.23
ElasticNet:
0.73
maxIter:
100
Trial loss:
0.1988924485125858

Trial Parameters:
regParam:
0.75
ElasticNet:
0.98
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.73
ElasticNet:
0.14
maxIter:
100
Trial loss:
0.07972704908108408

Trial Parameters:
regParam:
0.13
ElasticNet:
0.34
maxIter:
100
Trial loss:
0.07461575489995309

Trial Parameters:
regParam:
0.54
ElasticNet:
0.9500000000000001
maxIter:
100
Trial loss:
0.5

Trial Parameters:
regParam:
0.47000000000000003
ElasticNet:
0.08
maxIter:
100
Trial loss:
0.08197669768725557

Trial Parameters:
regParam:
0.44
ElasticNet:
0.49
maxIter:
100
Trial loss:
0.1988924485125858

Trial Parameters:
regParam:
0.46
ElasticNet:
0.17
maxIter:
100
Trial loss:
0.0809470324948024

Trial Parameters:
regParam:
0.35000000000000003
ElasticNet:
0.37
maxIter:
100
Trial loss:
0.07131314373045639

Trial Parameters:

In [ ]:
# Print best params
best_params

{'elasticNetParam': 0.88, 'regParam': 0.0}

In [ ]:
# Define vars to fit best params
best_elasticNetParam = best_params['elasticNetParam']
best_regParam = best_params['regParam']

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_elasticNetParam, best_regParam) 
print(f'The trained logistic regression achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
regParam: 0.88
ElasticNet: 0.0
maxIter: 100
The trained logistic regression achieved an AUROC score of 0.8866422519641158 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9860501025753575, and the final (tuned) model achieved 0.8866422519641158.


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9864204599207805, and the final (tuned) model achieved 0.8323038740761367.


In [ ]:
# Set up logistic regression pipeline: 
lr = LogisticRegression(labelCol='label', 
                        featuresCol='scaledFeatures', 
                        family='binomial', 
                        maxIter=100, 
                        regParam=0.0, 
                        elasticNetParam=0.0)

pipeline_lr = Pipeline(stages=[mmScaler, lr])
search_time_start = time.time()
pipelineModel_lr_hyperopt = pipeline_lr.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lr_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lr_hyperopt_smote_auroc_100trials')

2022/09/17 18:32:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e15612394d864f55b173c23aaf25d84f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 197.89344668388367


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hyperopt.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('Hyperopt: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

Hyperopt: Logistic Regression
Area under ROC curve: 0.98605
Test Error: 0.0139499 
Accuracy: 0.9866072564067584


### F1 Score

#### 100 Trials

In [ ]:
def train_model(regParam, elasticNetParam): 
  
  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # MinMaxScaler
    mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                            outputCol='scaledFeatures',
                            min=0, max=1)
    
    # Define model
    lr = LogisticRegression(labelCol='label', 
                            featuresCol='scaledFeatures',
                            family='binomial',                             
                            regParam=regParam,
                            elasticNetParam=elasticNetParam,
                            standardization=True)
    
    print('\nTrial Parameters:')
    print('regParam:', lr.getRegParam())
    print('ElasticNet:', lr.getElasticNetParam())
    print('maxIter:', lr.getMaxIter())

    # Create pipeline
    pipeline_lr_hpo = Pipeline(stages=[mmScaler, lr])
    
    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_lr_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, elasticNetParam=0.0)
print(f'The trained logistic regression achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
ElasticNet: 0.0
maxIter: 100


2022/09/09 16:07:28 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


The trained logistic regression achieved an F1 score of 0.9864204599207805 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  elasticNetParam = params['elasticNetParam']

  # Train model
  model, f1_score = train_model(regParam, elasticNetParam)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
  
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'elasticNetParam': hp.quniform('elasticNetParam', 0, 1, q=0.01),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=tpe.suggest,
      max_evals=100
  )


Trial Parameters:
regParam:
0.99
ElasticNet:
0.56
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.24
ElasticNet:
0.27
maxIter:
100
Trial loss:
0.11449618901955616

Trial Parameters:
regParam:
0.74
ElasticNet:
0.1
maxIter:
100
Trial loss:
0.11727264897041179

Trial Parameters:
regParam:
0.6
ElasticNet:
0.06
maxIter:
100
Trial loss:
0.13183272296111248

Trial Parameters:
regParam:
0.58
ElasticNet:
0.8
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.52
ElasticNet:
0.91
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.8300000000000001
ElasticNet:
0.26
maxIter:
100
Trial loss:
0.0785933247262336

Trial Parameters:
regParam:
0.07
ElasticNet:
0.22
maxIter:
100
Trial loss:
0.07423665985943806

Trial Parameters:
regParam:
0.53
ElasticNet:
0.27
maxIter:
100
Trial loss:
0.06724344068566934

Trial Parameters:
regParam:
0.25
ElasticNet:
1.0
maxIter:
100
Trial loss:
0.1852053697198861

Trial Parameters:
regParam:
0.77
E

In [ ]:
# Print best params
best_params

{'elasticNetParam': 0.01, 'regParam': 0.0}

In [ ]:
# Define vars to fit best params
best_elasticNetParam = best_params['elasticNetParam']
best_regParam = best_params['regParam']

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_elasticNetParam, best_regParam) 
print(f'The trained logistic regression achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
regParam: 0.01
ElasticNet: 0.0
maxIter: 100
The trained logistic regression achieved an F1 score of 0.961644148508058 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9864204599207805, and the final (tuned) model achieved 0.961644148508058.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9860501025753575, and the final (tuned) model achieved 0.9679372630414226.


In [ ]:
# Set up logistic regression pipeline
lr = LogisticRegression(labelCol='label', 
                        featuresCol='scaledFeatures', 
                        family='binomial', 
                        maxIter=100, 
                        regParam=0.0, 
                        elasticNetParam=0.0)

pipeline_lr = Pipeline(stages=[mmScaler, lr])
search_time_start = time.time()
pipelineModel_lr_hyperopt = pipeline_lr.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lr_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lr_hyperopt_smote_f1_100trials')

2022/10/08 19:29:54 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '426955ff4be7418d82d34c683ddaec20', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 309.7018527984619


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hyperopt.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('Hyperopt: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

Hyperopt: Logistic Regression
Area under ROC curve: 0.98605
Test Error: 0.0139499 
Accuracy: 0.9866072564067584


## LinearSVC

### AUROC Score

#### 100 Trials

In [ ]:
def train_model(regParam, tol, maxIter):
  
  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Standard scaler
    stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                               outputCol='scaledFeatures', 
                               withStd=True, 
                               withMean=False)
    
    # Define model
    lsvc = LinearSVC(labelCol='label', 
                     featuresCol='scaledFeatures',
                     regParam=regParam,
                     tol=tol,
                     maxIter=maxIter)

    print('\nTrial Parameters:')
    print('regParam:', lsvc.getRegParam())
    print('tol:', lsvc.getTol())
    print('maxIter:', lsvc.getMaxIter())

    # Create pipeline
    pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, lsvc])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_lsvc_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, tol=1e-5, maxIter=100)
print(f'The trained LinearSVC achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 1e-05
maxIter: 100


2022/09/11 15:28:24 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


The trained LinearSVC achieved an AUROC score of 0.9788742585676203 on the validation data


In [ ]:
def train_with_hyperopt(params):
  
  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  tol = params['tol']
  maxIter = int(params['maxIter'])

  # Train model
  model, AUROC_score = train_model(regParam, tol, maxIter)

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'tol': hp.quniform('tol', 0, 1e-3, q=5e-8),
  'maxIter': scope.int(hp.quniform('maxIter', 100, 1000, q=100)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=100
  )

                                                       
Trial Parameters:
regParam:                                              
0.27                                                   
tol:                                                   
0.000638                                               
maxIter:                                               
400                                                    
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                            
0.06659555353993885                                    
                                                                                    
Trial Parameters:
regParam:                                                                           
0.76                                                                                
tol:                                                                                
0.0009021999999999999                                                               
maxIter:                                                                            
900                                                                                 
  1%|          | 1/100 [01:19<2:09:51, 78.70s/trial, best loss: 0.06659555353993885]

Trial loss:                                                                         
0.10439184129203471                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.25                                                                                
tol:                                                                                
0.0001759                                                                           
maxIter:                                                                            
800                                                                                 
  2%|▏         | 2/100 [02:37<2:08:31, 78.68s/trial, best loss: 0.06659555353993885]

Trial loss:                                                                         
0.06430090346162265                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.88                                                                                
tol:                                                                                
0.0004911                                                                           
maxIter:                                                                            
200                                                                                 
  3%|▎         | 3/100 [04:24<2:27:59, 91.54s/trial, best loss: 0.06430090346162265]

Trial loss:                                                                         
0.11008762103410463                                                                 
                                                                                    
Trial Parameters:
regParam:                                                                           
0.31                                                                                
tol:                                                                                
9.515e-05                                                                           
maxIter:                                                                            
100                                                                                 
  4%|▍         | 4/100 [05:43<2:18:36, 86.63s/trial, best loss: 0.06430090346162265]

...

Trial loss:                                                                          
0.05155082188437787                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.24                                                                                 
tol:                                                                                 
0.0009069                                                                            
maxIter:                                                                             
500                                                                                  
 96%|█████████▌| 96/100 [2:36:09<06:02, 90.51s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.06313753535801336                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.9400000000000001                                                                   
tol:                                                                                 
0.0009406999999999999                                                                
maxIter:                                                                             
400                                                                                  
 97%|█████████▋| 97/100 [2:37:28<04:21, 87.18s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.11262167172716175                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.1                                                                                  
tol:                                                                                 
0.00035184999999999996                                                               
maxIter:                                                                             
200                                                                                  
 98%|█████████▊| 98/100 [2:38:46<02:48, 84.26s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.04420100533688376                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.06                                                                                 
tol:                                                                                 
0.0007159499999999999                                                                
maxIter:                                                                             
300                                                                                  
 99%|█████████▉| 99/100 [2:40:13<01:25, 85.15s/trial, best loss: 0.01981533806748803]

Trial loss:                                                                          
0.037522653304294384                                                                 
100%|██████████| 100/100 [2:41:40<00:00, 97.01s/trial, best loss: 0.01981533806748803]


In [ ]:
# Print best params
best_params

{'maxIter': 500.0, 'regParam': 0.0, 'tol': 0.0009712999999999999}

In [ ]:
# Define vars to fit best params
best_regParam = best_params['regParam']
best_tol = best_params['tol']
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_auroc_score = train_model(best_regParam, best_tol, best_maxIter)
print(f'The trained LinearSVC achieved an AUROC score of {val_auroc_score} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 0.0009712999999999999
maxIter: 500


The trained LinearSVC achieved an AUROC score of 0.980184661932512 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9788742585676203, and the final (tuned) model achieved 0.980184661932512.


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9797006472779239, and the final (tuned) model achieved 0.9809885079539138.


In [ ]:
# Set up LinearSVC pipeline: {'maxIter': 500.0, 'regParam': 0.0, 'tol': 0.0009712999999999999}
lsvc = LinearSVC(labelCol='label',
                 featuresCol='scaledFeatures',
                 regParam=best_regParam,
                 tol=best_tol,
                 maxIter=best_maxIter)

pipeline_lsvc = Pipeline(stages=[stdScaler, lsvc])
search_time_start = time.time()
pipelineModel_lsvc_hyperopt = pipeline_lsvc.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lsvc_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lsvc_hyperopt_smote_auroc')

2022/10/09 02:43:28 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cd765942dc0445c2893b952c72c91beb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 02:49:46 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit best model: 414.80314660072327


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hyperopt.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('Hyperopt: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lsvc)) 

Hyperopt: LinearSVC
Area under ROC curve: 0.979253
Test Error: 0.0207471
Accuracy: 0.9824312731661861


### F1 Score

#### 100 Trials

In [ ]:
def train_model(regParam, tol, maxIter):
  
  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Standard scaler
    stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                               outputCol='scaledFeatures', 
                               withStd=True, 
                               withMean=False)
    
    # Define model
    lsvc = LinearSVC(labelCol='label', 
                     featuresCol='scaledFeatures',
                     regParam=regParam,
                     tol=tol,
                     maxIter=maxIter)

    print('\nTrial Parameters:')
    print('regParam:', lsvc.getRegParam())
    print('tol:', lsvc.getTol())
    print('maxIter:', lsvc.getMaxIter())

    # Create pipeline
    pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, lsvc])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')
    # Fit model
    model = pipeline_lsvc_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
initial_model, val_metric = train_model(regParam=0.0, tol=1e-5, maxIter=100)
print(f'The trained LinearSVC achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 1e-05
maxIter: 100


2022/09/10 05:07:32 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


The trained LinearSVC achieved an F1 score of 0.9816118037353824 on the validation data


In [ ]:
def train_with_hyperopt(params):
  
  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  regParam = params['regParam']
  tol = params['tol']
  maxIter = int(params['maxIter'])

  # Train model
  model, f1_score = train_model(regParam, tol, maxIter)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'regParam': hp.quniform('regParam',  0, 1, q=0.01),
  'tol': hp.quniform('tol', 0, 1e-3, q=5e-8),
  'maxIter': scope.int(hp.quniform('maxIter', 100, 1000, q=100)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=100
  )

                                                       
Trial Parameters:
regParam:                                              
0.96                                                   
tol:                                                   
0.00017245                                             
maxIter:                                               
400                                                    
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                            
0.1706129670964539                                     
                                                                                   
Trial Parameters:
regParam:                                                                          
0.68                                                                               
tol:                                                                               
0.00054385                                                                         
maxIter:                                                                           
300                                                                                
  1%|          | 1/100 [01:14<2:02:59, 74.54s/trial, best loss: 0.1706129670964539]

Trial loss:                                                                        
0.1544083638848972                                                                 
                                                                                   
Trial Parameters:
regParam:                                                                          
0.02                                                                               
tol:                                                                               
0.00024065                                                                         
maxIter:                                                                           
500                                                                                
  2%|▏         | 2/100 [02:29<2:01:32, 74.41s/trial, best loss: 0.1544083638848972]

Trial loss:                                                                        
0.028085823670832566                                                               
                                                                                     
Trial Parameters:
regParam:                                                                            
0.12                                                                                 
tol:                                                                                 
0.0005275999999999999                                                                
maxIter:                                                                             
200                                                                                  
  3%|▎         | 3/100 [04:02<2:14:09, 82.98s/trial, best loss: 0.028085823670832566]

Trial loss:                                                                          
0.06447643711062623                                                                  
                                                                                     
Trial Parameters:
regParam:                                                                            
0.13                                                                                 
tol:                                                                                 
0.00037515                                                                           
maxIter:                                                                             
600                                                                                  
  4%|▍         | 4/100 [05:20<2:09:52, 81.17s/trial, best loss: 0.028085823670832566]

...

Trial loss:                                                                           
0.15277896738414232                                                                   
                                                                                      
Trial Parameters:
regParam:                                                                             
0.35000000000000003                                                                   
tol:                                                                                  
2.51e-05                                                                              
maxIter:                                                                              
600                                                                                   
 96%|█████████▌| 96/100 [3:06:27<05:39, 84.86s/trial, best loss: 0.016121584912915154]

Trial loss:                                                                           
0.12004464934605974                                                                   
                                                                                      
Trial Parameters:
regParam:                                                                             
0.05                                                                                  
tol:                                                                                  
0.0006213                                                                             
maxIter:                                                                              
800                                                                                   
 97%|█████████▋| 97/100 [3:07:48<04:11, 83.97s/trial, best loss: 0.016121584912915154]

Trial loss:                                                                           
0.03533423331246821                                                                   
                                                                                      
Trial Parameters:
regParam:                                                                             
0.24                                                                                  
tol:                                                                                  
0.00083125                                                                            
maxIter:                                                                              
700                                                                                   
 98%|█████████▊| 98/100 [3:09:19<02:52, 86.42s/trial, best loss: 0.016121584912915154]

Trial loss:                                                                           
0.09969280215906662                                                                   
                                                                                      
Trial Parameters:
regParam:                                                                             
0.29                                                                                  
tol:                                                                                  
0.00010059999999999999                                                                
maxIter:                                                                              
300                                                                                   
 99%|█████████▉| 99/100 [3:10:35<01:23, 83.49s/trial, best loss: 0.016121584912915154]

Trial loss:                                                                           
0.11000165234476222                                                                   
100%|██████████| 100/100 [3:11:53<00:00, 115.13s/trial, best loss: 0.016121584912915154]


In [ ]:
# Print best params
best_params

{'maxIter': 700.0, 'regParam': 0.0, 'tol': 0.0001082}

In [ ]:
# Define vars to fit best params
best_regParam = best_params['regParam']
best_tol = best_params['tol']
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_regParam, best_tol, best_maxIter)
print(f'The trained LinearSVC achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
regParam: 0.0
tol: 0.0001082
maxIter: 700


The trained LinearSVC achieved an F1 score of 0.9838784150870848 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9816118037353824, and the final (tuned) model achieved 0.9838784150870848.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.979112214992606, and the final (tuned) model achieved 0.981959571141341.


In [ ]:
# Set up LinearSVC pipeline: {'maxIter': 700.0, 'regParam': 0.0, 'tol': 0.0001082}
lsvc = LinearSVC(labelCol='label',
                 featuresCol='scaledFeatures',
                 regParam=best_regParam,
                 tol=best_tol,
                 maxIter=best_maxIter)

pipeline_lsvc = Pipeline(stages=[stdScaler, lsvc])
search_time_start = time.time()
pipelineModel_lsvc_hyperopt = pipeline_lsvc.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_lsvc_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_lsvc_hyperopt_smote_f1')

2022/09/17 18:57:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'efce1a6ca8764205b0faa91a1f6401ee', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow


Time to fit best model: 984.2158923149109


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hyperopt.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('Hyperopt: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lsvc)) 

Hyperopt: LinearSVC
Area under ROC curve: 0.98196
Test Error: 0.0180404
Accuracy: 0.9841793591738675


## Decision Tree

### AUROC Score

##### 50 Trials 

In [ ]:
def train_model(maxDepth, maxBins):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    dtc = DecisionTreeClassifier(labelCol='label', 
                                 featuresCol='unscaledFeatures',
                                 maxDepth=maxDepth,
                                 maxBins=maxBins,
                                 seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', dtc.getMaxDepth())
    print('maxBins:', dtc.getMaxBins())  
    
    # Create pipeline
    pipeline_dt_hpo = Pipeline(stages=[dtc])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_dt_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=16)
print('Time to fit model:', time.time() - search_time_start)
print(f'The trained decision tree classifier achieved an AUROC score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 16


2022/09/14 17:52:07 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


Time to fit model: 103.11991286277771
The trained decision tree classifier achieved an AUROC score of 0.9207046679916379 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins)
  
  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)
    
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 30, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 128, q=16)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin(
    fn=train_with_hyperopt,
    space=space,
    algo=algo,
    max_evals=50
  )

                                                      
Trial Parameters:
maxDepth:                                             
22                                                    
maxBins:                                              
80                                                    
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                           
0.03929887269090371                                   
                                                                                    
Trial Parameters:
maxDepth:                                                                           
10                                                                                  
maxBins:                                                                            
128                                                                                 
  2%|▏         | 1/50 [02:10<1:46:18, 130.18s/trial, best loss: 0.03929887269090371]

Trial loss:                                                                         
0.058707586531108635                                                                
                                                                                    
Trial Parameters:
maxDepth:                                                                          
17                                                                                 
maxBins:                                                                           
48                                                                                 
  4%|▍         | 2/50 [03:22<1:16:23, 95.50s/trial, best loss: 0.03929887269090371]

Trial loss:                                                                        
0.04461781515233543                                                                
                                                                                   
Trial Parameters:
maxDepth:                                                                          
29                                                                                 
maxBins:                                                                           
80                                                                                 
  6%|▌         | 3/50 [04:48<1:11:48, 91.68s/trial, best loss: 0.03929887269090371]

2022/09/14 18:00:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpaor0it3x/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



...

Trial loss:                                                                          
0.03856738072050281                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
30                                                                                   
maxBins:                                                                             
128                                                                                  
 96%|█████████▌| 48/50 [1:43:12<04:35, 137.97s/trial, best loss: 0.03802398165072818]

2022/09/14 19:39:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpwtt32vgi/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.04059547403392072                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
25                                                                                   
maxBins:                                                                             
48                                                                                   
 98%|█████████▊| 49/50 [1:47:17<02:49, 169.99s/trial, best loss: 0.03802398165072818]

2022/09/14 19:42:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpoq13tu1z/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.04149484538405068                                                                  
100%|██████████| 50/50 [1:49:46<00:00, 131.72s/trial, best loss: 0.03802398165072818]


In [ ]:
# Print best params
best_params

{'maxBins': 80.0, 'maxDepth': 26.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins)
print(f'The trained decision tree classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 26
maxBins: 80


2022/09/14 19:45:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpw71_lfl7/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.


The trained decision tree classifier achieved an AUROC score of 0.9619760183492718 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9207046679916379, and the final (tuned) model achieved 0.9619760183492718.


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9462684208910764, and the final (tuned) model achieved 0.9804645436906474.


In [ ]:
# Set up DecisionTree pipeline: {'maxBins': 80.0, 'maxDepth': 26.0}
dt = DecisionTreeClassifier(labelCol='label', 
                            featuresCol='unscaledFeatures', 
                            impurity='gini', 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_dt = Pipeline(stages=[dt])
pipelineModel_dt_hyperopt = pipeline_dt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_dt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_dt_hyperopt_smote_auroc')

2022/09/17 19:15:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'da7a8b1326a34aeca62dadaaf935596e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/09/17 19:20:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppev9yazf/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 262.11992478370667


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hyperopt.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('Hyperopt: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

Hyperopt: Decision Tree
Area under ROC curve: 0.961976
Test Error: 0.038024 
Accuracy: 0.980546762456847


### F1 Score

##### 50 Trials 

In [ ]:
def train_model(maxDepth, maxBins):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    dtc = DecisionTreeClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', dtc.getMaxDepth())
    print('maxBins:', dtc.getMaxBins())    
    
    # Create pipeline
    pipeline_dt_hpo = Pipeline(stages=[dtc])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_dt_hpo.fit(trainDF)   
    
    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=16)
print('Time to fit model:', time.time() - search_time_start)
print(f'The trained decision tree classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 16


Time to fit model: 57.041776180267334
The trained decision tree classifier achieved an F1 score of 0.9462684208910764 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins)
  
  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)
    
  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 30, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 128, q=16)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin(
    fn=train_with_hyperopt,
    space=space,
    algo=algo,
    max_evals=50
  )

                                                      
Trial Parameters:
maxDepth:                                             
7                                                     
maxBins:                                              
32                                                    
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                           
0.03398379755296532                                   
                                                                                 
Trial Parameters:
maxDepth:                                                                        
21                                                                               
maxBins:                                                                         
48                                                                               
  2%|▏         | 1/50 [00:59<47:39, 58.36s/trial, best loss: 0.03398379755296532]

Trial loss:                                                                      
0.01605558638521265                                                              
                                                                                   
Trial Parameters:
maxDepth:                                                                          
24                                                                                 
maxBins:                                                                           
64                                                                                 
  4%|▍         | 2/50 [02:39<1:06:40, 83.35s/trial, best loss: 0.01605558638521265]

2022/09/14 13:36:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpn5rspcjk/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                        
0.018297493565652134                                                               
                                                                                    
Trial Parameters:
maxDepth:                                                                           
11                                                                                  
maxBins:                                                                            
96                                                                                  
  6%|▌         | 3/50 [05:05<1:27:38, 111.89s/trial, best loss: 0.01605558638521265]

...

Trial loss:                                                                          
0.014181856639880475                                                                 
                                                                                     
Trial Parameters:
maxDepth:                                                                            
22                                                                                   
maxBins:                                                                             
112                                                                                  
 94%|█████████▍| 47/50 [1:18:49<04:34, 91.47s/trial, best loss: 0.013950227135559934]

Trial loss:                                                                          
0.01600105843886268                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
25                                                                                    
maxBins:                                                                              
128                                                                                   
 96%|█████████▌| 48/50 [1:21:01<03:32, 106.40s/trial, best loss: 0.013950227135559934]

2022/09/14 14:55:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpe19riy0f/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.017962752633492785                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
10                                                                                    
maxBins:                                                                              
32                                                                                    
 98%|█████████▊| 49/50 [1:24:02<02:10, 130.47s/trial, best loss: 0.013950227135559934]

Trial loss:                                                                           
0.02347456757207389                                                                   
100%|██████████| 50/50 [1:25:04<00:00, 102.10s/trial, best loss: 0.013950227135559934]


In [ ]:
# Print best params
best_params

{'maxBins': 128.0, 'maxDepth': 17.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins)
print(f'The trained decision tree classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 17
maxBins: 128


The trained decision tree classifier achieved an F1 score of 0.9860497728644401 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9462684208910764, and the final (tuned) model achieved 0.9860497728644401.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9207046679916379, and the final (tuned) model achieved 0.9494460525988754.


In [ ]:
# Set up DecisionTree pipeline: {'maxBins': 128.0, 'maxDepth': 17.0}
dt = DecisionTreeClassifier(labelCol='label', 
                            featuresCol='unscaledFeatures', 
                            impurity='gini', 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_dt = Pipeline(stages=[dt])
pipelineModel_dt_hyperopt = pipeline_dt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_dt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_dt_hyperopt_smote_f1')

Time to fit best model: 148.977787733078


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hyperopt.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('Hyperopt: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '%(1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

Hyperopt: Decision Tree
Area under ROC curve: 0.949446
Test Error: 0.0505539 
Accuracy: 0.9862858490587851


## Random Forest

### AUROC Score

##### 30 Trials 

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', rf.getMaxDepth())
    print('maxBins:', rf.getMaxBins())	    
    print('numTrees:', rf.getNumTrees())

    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print(f'The trained random forest classifier achieved an AUROC score of {val_metric} on the validation data')

2022/09/13 19:43:54 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


The trained random forest classifier achieved an AUROC score of 0.9587984522925381 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins, numTrees) 

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss: ', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 10, 30, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )


Trial Parameters:
maxDepth:
17
maxBins:
36
numTrees:
14
  0%|          | 0/30 [00:00<?, ?it/s, best loss: ?] 2022/09/14 00:47:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpaf_cs9vq/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.021961084153946597

Trial Parameters:
maxDepth:
13
maxBins:
8
numTrees:
27
  3%|▎         | 1/30 [12:46<6:10:08, 765.82s/it, best loss: 0.021961084153946597] 2022/09/14 01:00:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcsdex4qh/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.025483678252661734

Trial Parameters:
maxDepth:
6
maxBins:
56
numTrees:
24

Trial Loss:
0.046085695302414575

Trial Parameters:
maxDepth:
17
maxBins:
64
numTrees:
21
 10%|█

In [ ]:
# Print best params
best_params

{'maxBins': 36.0, 'maxDepth': 20.0, 'numTrees': 30.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params
final_model, val_auroc_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained random forest classifier achieved an AUROC score of {val_auroc_score} on the validation data')


Trial Parameters:
maxDepth: 20
maxBins: 36
numTrees: 30
2022/09/14 11:34:13 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmptny8vzlj/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained random forest classifier achieved an AUROC score of 0.9808695244711634 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9587984522925381, and the final (tuned) model achieved 0.9808695244711634.


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.testDF')

On the test data, the initial (untuned) model achieved F1 score 0.9567644210793013, and the final (tuned) model achieved 0.9814159616320126.


In [ ]:
# Set up RF pipeline {'maxBins': 36.0, 'maxDepth': 20.0, 'numTrees': 30.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_smote_auroc_30trials')

2022/10/09 16:56:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cfec6e7a899e446089ab1355b794f800', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 17:32:37 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/10/09 17:33:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmptsl6dm1y/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 2266.181292772293


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.980303
Test Error: 0.0196975
Accuracy: 0.9816242863716348


#### 20 Trials - Increased Trees & Depth 

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 10, 30, q=5)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 32, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 20, 50, q=5)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=20
  )


Trial Parameters:
maxDepth:
25
maxBins:
20
numTrees:
45
 0%|          | 0/20 [00:00<?, ?it/s, best loss: ?] 2022/09/16 03:12:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppr4k0686/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.01938575256515951

Trial Parameters:
maxDepth:
20
maxBins:
16
numTrees:
35
 5%|▌         | 1/20 [1:16:20<24:10:14, 4579.73s/it, best loss: 0.01938575256515951] 2022/09/16 03:50:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp7qjb4s3q/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial Loss:
0.0195991711848873

Trial Parameters:
maxDepth:
10
maxBins:
32
numTrees:
35
 10%|█         | 2/20 [1:53:09<15:55:40, 3185.57s/it, best loss: 0.01938575256515951] 2022/09/16 

In [ ]:
# Print best params
best_params

{'maxBins': 28.0, 'maxDepth': 30.0, 'numTrees': 50.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params and evaluate AUROC tuned model
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained Random Forest classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 30
maxBins: 28
numTrees: 50
2022/09/17 03:44:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ub7jxeq/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an AUROC score of 0.9813288878358067 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9567644210793013, and the final (tuned) model achieved 0.9834082187901869.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9587984522925381, and the final (tuned) model achieved 0.9813288878358067.


In [ ]:
# Set up RF pipeline: {'maxBins': 28.0, 'maxDepth': 30.0, 'numTrees': 50.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_smote_auroc_moreParams')

2022/10/09 17:35:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3b68c75458224f07b079f4147aecf252', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 20:35:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpb_tcxjkt/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 10772.246058702469


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.980929
Test Error: 0.0190706 
Accuracy: 0.9837770219181313


### F1 Score

#### 30 Trials 

In [ ]:
def train_model(maxDepth, maxBins, numTrees):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    rf = RandomForestClassifier(labelCol='label', 
                                featuresCol='unscaledFeatures',
                                maxDepth=maxDepth,
                                maxBins=maxBins,
                                numTrees=numTrees,
                                seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', rf.getMaxDepth())
    print('maxBins:', rf.getMaxBins())	    
    print('numTrees:', rf.getNumTrees())

    # Create pipeline
    pipeline_rf_hpo = Pipeline(stages=[rf])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_rf_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, numTrees=10) 
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained Random Forest classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 32
numTrees: 10


2022/09/14 20:24:53 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to fit initial model: 199.50931787490845
The trained Random Forest classifier achieved an F1 score of 0.9567644210793013 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  numTrees = int(params['numTrees'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, numTrees)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 10, 30, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )


Trial Parameters:
maxDepth:
12
maxBins:
12
numTrees:
14
  0%|          | 0/30 [00:00<?, ?it/s, best loss: ?] 2022/09/14 13:06:07 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ur7j4g6/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial loss:
0.03729664720677428

Trial Parameters:
maxDepth:
16
maxBins:
60
numTrees:
21
3%|▎         | 1/30 [05:23<2:36:14, 323.25s/it, best loss: 0.03729664720677428] 2022/09/14 13:23:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8y_sujuz/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial loss:
0.026438171428926793

Trial Parameters:
maxDepth:
14
maxBins:
64
numTrees:
25
7%|▋         | 2/30 [22:46<5:48:28, 746.73s/it, best loss: 0.026438171428926793] 2022/09/14 13:38:2

In [ ]:
# Print best params
best_params

{'maxBins': 36.0, 'maxDepth': 20.0, 'numTrees': 17.0}


In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained Random Forest classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 20
maxBins: 36
numTrees: 17
2022/09/14 20:43:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpvu3ivtay/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an F1 score of 0.9809512606581499 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9567644210793013, and the final (tuned) model achieved 0.9809512606581499


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9587984522925381, and the final (tuned) model achieved 0.9805702355001202


In [ ]:
# Set up RF pipeline: {'maxBins': 36.0, 'maxDepth': 20.0, 'numTrees': 17.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            numTrees=best_numTrees,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_smote_f1_29trials')

2022/10/09 17:31:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '780d019fb5364e6abed891b5fbf18d13', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 17:50:53 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpjamu7fb1/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 1177.357032775879


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.98057
Test Error: 0.0194298 
Accuracy: 0.9815433564638717


#### 20 Trials - Increased Trees

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 10, 30, q=5)),
  'maxBins': scope.int(hp.quniform('maxBins', 16, 32, q=4)),
  'numTrees': scope.int(hp.quniform('numTrees', 20, 50, q=5)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params=fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=20
  )

  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?] 2022/09/11 05:40:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzgre5r4l/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial loss:
0.019860350200941013
  5%|▌         | 1/20 [37:53<12:00:01, 2273.79s/it, best loss: 0.019860350200941013] 2022/09/11 06:25:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzkrfjmzz/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.

Trial loss:
0.018801600979486244
 10%|█         | 2/20 [1:22:56<12:37:54, 2526.35s/it, best loss: 0.018801600979486244] 2022/09/11 06:51:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpq2w_k2jw/model, flavor: spark), fall b

In [ ]:
# Print best params
best_params

{'maxBins': 28.0, 'maxDepth': 25.0, 'numTrees': 30.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_numTrees = int(best_params['numTrees'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, best_numTrees)
print(f'The trained Random Forest classifier achieved an F1 score of {val_f1_score} on the validation data')

2022/09/11 18:33:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmphspx9d7_/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
The trained Random Forest classifier achieved an F1 score of 0.9829400045609562 on the validation data


In [ ]:
# Evaluate F1 on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.testDF')

On the test data, the initial (untuned) model achieved F1 score 0.9567644210793013, and the final (tuned) model achieved 0.9829400045609562.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9587984522925381, and the final (tuned) model achieved 0.980338569643614.


In [ ]:
# Set up RF pipeline: {'maxBins': 28.0, 'maxDepth': 25.0, 'numTrees': 30.0}
rf = RandomForestClassifier(labelCol='label',  
                            featuresCol='unscaledFeatures',  
                            impurity='gini',  
                            numTrees=best_numTrees, 
                            maxDepth=best_maxDepth, 
                            maxBins=best_maxBins,
                            seed=seed_value)

search_time_start = time.time()
pipeline_rf = Pipeline(stages=[rf])
pipelineModel_rf_hyperopt = pipeline_rf.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_rf_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_rf_hyperopt_smote_f1_moreParams')

2022/10/09 18:20:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ac15fe09fc0c414a9d21f96c6fa03e15', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 19:23:36 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/10/09 19:25:46 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp0ixp3ehu/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 3929.8262758255005


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hyperopt.transform(testDF)

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('Hyperopt: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

Hyperopt: Random Forest
Area under ROC curve: 0.980243
Test Error: 0.0197569
Accuracy: 0.9828821683665802


## Gradient Boosted Trees

### AUROC Score

#### 30 Trials

In [ ]:
def train_model(maxDepth, maxBins, maxIter):

  """
   - Input: Hyperparameters for tuning
   - Return: AUROC score on the validation set
  """
 
  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    gbt = GBTClassifier(labelCol='label', 
                        featuresCol='unscaledFeatures',
                        maxDepth=maxDepth,
                        maxBins=maxBins,
                        maxIter=maxIter,
                        seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', gbt.getMaxDepth())
    print('maxBins:', gbt.getMaxBins())
    print('maxIter:', gbt.getMaxIter())
    
    # Create pipeline
    pipeline_gbt_hpo = Pipeline(stages=[gbt])

    # Define an evaluation metric
    evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                    metricName='areaUnderROC')

    # Fit model
    model = pipeline_gbt_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_auroc.evaluate(predictions)

    mlflow.log_metric('val_AUROC_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model 
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, maxIter=10)
print(f'The trained gradient boosted tree classifier achieved an AUROC score of {val_metric} on the validation data')

Trial Parameters:
maxDepth: 5
maxBins: 32
maxIter: 10
2022/09/14 12:45:36 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.
Time to fit initial model: 168.4599220752716
The trained gradient boosted tree classifier achieved an AUROC score of 0.9709812704945415 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  maxIter = int(params['maxIter'])

  # Train model
  model, AUROC_score = train_model(maxDepth, maxBins, maxIter)

  # Return a loss to monitor
  loss = 1 - AUROC_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'maxIter': scope.int(hp.quniform('maxIter', 5, 20, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )

                                                      
Trial Parameters:
maxDepth:                                             
11                                                    
maxBins:                                              
60                                                    
maxIter:                                              
11                                                    
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2022/09/14 05:26:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp3ofze7uk/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                           
0.014532307253502297                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
12                                                                                   
maxBins:                                                                             
24                                                                                   
maxIter:                                                                             
8                                                                                    
  3%|▎         | 1/30 [04:47<2:18:29, 286.53s/trial, best loss: 0.014532307253502297]

2022/09/14 05:29:53 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpxg7ns5pb/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.01451528736968255                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                           
15                                                                                  
maxBins:                                                                            
44                                                                                  
maxIter:                                                                            
19                                                                                  
  7%|▋         | 2/30 [08:27<1:55:25, 247.35s/trial, best loss: 0.01451528736968255]

...

Trial loss:                                                                          
0.013219048939419609                                                                 
                                                                                     
Trial Parameters:
maxDepth:                                                                            
14                                                                                   
maxBins:                                                                             
40                                                                                   
maxIter:                                                                             
20                                                                                   
 93%|█████████▎| 28/30 [4:36:09<28:15, 847.80s/trial, best loss: 0.01287452535609701]

2022/09/14 10:10:04 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpi00fa8oy/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.01287452535609701                                                                  
                                                                                     
Trial Parameters:
maxDepth:                                                                            
11                                                                                   
maxBins:                                                                             
16                                                                                   
maxIter:                                                                             
16                                                                                   
 97%|█████████▋| 29/30 [4:48:43<13:39, 819.62s/trial, best loss: 0.01287452535609701]

2022/09/14 10:15:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpfqbfxfgn/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                          
0.015939107256428664                                                                 
100%|██████████| 30/30 [4:54:18<00:00, 588.61s/trial, best loss: 0.01287452535609701]


In [ ]:
# Print best params
best_params

{'maxDepth': 14.0, 'maxBins': 40.0, 'maxIter': 20.0}


In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_AUROC_score = train_model(best_maxDepth, best_maxBins, 
                                           best_maxIter)
print(f'The trained gradient booster tree classifier achieved an AUROC score of {val_AUROC_score} on the validation data')


Trial Parameters:
maxDepth: 14
maxBins: 40
maxIter: 20


2022/09/14 12:58:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp5_091a0_/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.


The trained gradient booster tree classifier achieved an AUROC score of 0.987125474643903 on the validation data


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9709812704945415, and the final (tuned) model achieved 0.987125474643903.


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9646490135219191, and the final (tuned) model achieved 0.9863474688529502.


In [ ]:
# Set up GBT pipeline {'maxDepth': 14.0, 'maxBins': 40.0, 'maxIter': 20.0}
gbt = GBTClassifier(labelCol='label', 
                    featuresCol='unscaledFeatures', 
                    maxDepth=best_maxDepth, 
                    maxBins=best_maxBins, 
                    maxIter=best_maxIter,
                    seed=seed_value)

search_time_start = time.time()
pipeline_gbt = Pipeline(stages=[gbt])
pipelineModel_gbt_hyperopt = pipeline_gbt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_gbt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_gbt_hyperopt_smote_auroc')

2022/10/09 16:44:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6250aaa381e547d585c4c668fc75ee3e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 17:01:34 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/10/09 17:02:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpmul3yh9m/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 1093.5795516967773


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hyperopt.transform(testDF) 

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('Hyperopt: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

Hyperopt: Gradient Boosted Trees
Area under ROC curve: 0.987125
Test Error: 0.0128746 
Accuracy: 0.9866095686898373


### F1 Score

##### 30 Trials

In [ ]:
def train_model(maxDepth, maxBins, maxIter):

  """
   - Input: Hyperparameters for tuning
   - Return: F1 score on the validation set
  """

  # Use MLflow to track with 'nested=True' logged as a child run 
  with mlflow.start_run(nested=True):

    # Define model
    gbt = GBTClassifier(labelCol='label', 
                        featuresCol='unscaledFeatures',
                        maxDepth=maxDepth,
                        maxBins=maxBins,
                        maxIter=maxIter,
                        seed=seed_value)

    print('\nTrial Parameters:')
    print('maxDepth:', gbt.getMaxDepth())
    print('maxBins:', gbt.getMaxBins())
    print('maxIter:', gbt.getMaxIter())
    
    # Create pipeline
    pipeline_gbt_hpo = Pipeline(stages=[gbt])

    # Define an evaluation metric
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol='label', 
                                                     metricName='f1')

    # Fit model
    model = pipeline_gbt_hpo.fit(trainDF)   

    # Evaluate the model on the validation dataset
    predictions = model.transform(testDF)
    validation_metric = evaluator_f1.evaluate(predictions)

    mlflow.log_metric('val_f1_score', validation_metric)
 
  return model, validation_metric

In [ ]:
# Run initial model
search_time_start = time.time()
initial_model, val_metric = train_model(maxDepth=5, maxBins=32, maxIter=10)
print('Time to fit initial model:', time.time() - search_time_start)
print(f'The trained gradient boosted tree classifier achieved an F1 score of {val_metric} on the validation data')


Trial Parameters:
maxDepth: 5
maxBins: 32
maxIter: 10


2022/09/16 15:40:52 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.


Time to fit initial model: 170.6928231716156
The trained gradient boosted tree classifier achieved an F1 score of 0.9646490135219191 on the validation data


In [ ]:
def train_with_hyperopt(params):

  """
  Train method where hyperparameters are passed to hyperopt.fmin()
  - Input: Hyperparameters (params) in dictionary
  - Return: Dictionary with 'loss' and 'status' (success/failure of run)
  """

  # Convert integer params to int type if searching over continuous values
  maxDepth = int(params['maxDepth'])
  maxBins = int(params['maxBins'])
  maxIter = int(params['maxIter'])

  # Train model
  model, f1_score = train_model(maxDepth, maxBins, maxIter)

  # Return a loss to monitor
  loss = 1 - f1_score
  print('Trial loss:', loss)

  return {'loss': loss, 'status': STATUS_OK}

In [ ]:
# Define parameter space for HPO
space = {
  'maxDepth': scope.int(hp.quniform('maxDepth', 5, 20, q=1)),
  'maxBins': scope.int(hp.quniform('maxBins', 4, 64, q=4)),
  'maxIter': scope.int(hp.quniform('maxIter', 5, 20, q=1)),
}

In [ ]:
# Run HPO
algo=tpe.suggest

with mlflow.start_run():
    
  best_params = fmin( 
      fn=train_with_hyperopt,
      space=space,
      algo=algo,
      max_evals=30
  )

                                                      
Trial Parameters:
maxDepth:                                             
9                                                     
maxBins:                                              
44                                                    
maxIter:                                              
16                                                    
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

Trial loss:                                           
0.0151035551560188                                    
                                                                                   
Trial Parameters:
maxDepth:                                                                          
13                                                                                 
maxBins:                                                                           
20                                                                                 
maxIter:                                                                           
6                                                                                  
  3%|▎         | 1/30 [04:42<2:16:15, 281.92s/trial, best loss: 0.0151035551560188]

2022/09/16 15:49:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpbdhjfe0c/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                        
0.015776764343602867                                                               
                                                                                   
Trial Parameters:
maxDepth:                                                                          
11                                                                                 
maxBins:                                                                           
8                                                                                  
maxIter:                                                                           
10                                                                                 
  7%|▋         | 2/30 [07:57<1:47:39, 230.69s/trial, best loss: 0.0151035551560188]

2022/09/16 15:52:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmppz1jq22c/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



...

Trial loss:                                                                           
0.013893179504413267                                                                  
                                                                                      
Trial Parameters:
maxDepth:                                                                             
10                                                                                    
maxBins:                                                                              
40                                                                                    
maxIter:                                                                              
15                                                                                    
 93%|█████████▎| 28/30 [4:37:17<25:57, 778.85s/trial, best loss: 0.013514999688791773]

2022/09/16 20:23:19 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpgbul9g1h/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.01492893992191846                                                                   
                                                                                      
Trial Parameters:
maxDepth:                                                                             
14                                                                                    
maxBins:                                                                              
48                                                                                    
maxIter:                                                                              
16                                                                                    
 97%|█████████▋| 29/30 [4:42:11<10:33, 633.36s/trial, best loss: 0.013514999688791773]

2022/09/16 20:33:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprarjxi3m/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.



Trial loss:                                                                           
0.013641007246408487                                                                  
100%|██████████| 30/30 [4:52:04<00:00, 584.14s/trial, best loss: 0.013514999688791773]


In [ ]:
# Print best params
best_params

{'maxBins': 52.0, 'maxDepth': 13.0, 'maxIter': 17.0}

In [ ]:
# Define vars to fit best params
best_maxDepth = int(best_params['maxDepth'])
best_maxBins = int(best_params['maxBins'])
best_maxIter = int(best_params['maxIter'])

In [ ]:
# Fit model with best params
final_model, val_f1_score = train_model(best_maxDepth, best_maxBins, best_maxIter)
print(f'The trained gradient booster tree classifier achieved an F1 score of {val_f1_score} on the validation data')


Trial Parameters:
maxDepth: 13
maxBins: 52
maxIter: 17


2022/09/16 20:42:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpr1awxjh7/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.


The trained gradient booster tree classifier achieved an F1 score of 0.9864850003112082 on the validation data


In [ ]:
# Evaluate F1 score on initial and tuned model
initial_model_test_metric = evaluator_f1.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_f1.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved F1 score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved F1 score 0.9646490135219191, and the final (tuned) model achieved 0.9864850003112082.


In [ ]:
# Evaluate AUROC on initial and tuned model
initial_model_test_metric = evaluator_auroc.evaluate(initial_model.transform(testDF))
final_model_test_metric = evaluator_auroc.evaluate(final_model.transform(testDF))
print(f'On the test data, the initial (untuned) model achieved AUROC score {initial_model_test_metric}, and the final (tuned) model achieved {final_model_test_metric}.')

On the test data, the initial (untuned) model achieved AUROC score 0.9709812704945415, and the final (tuned) model achieved 0.9869669643047061.


In [ ]:
# Set up GBT pipeline {'maxBins': 52.0, 'maxDepth': 13.0, 'maxIter': 17.0}
gbt = GBTClassifier(labelCol='label', 
                    featuresCol='unscaledFeatures', 
                    maxDepth=best_maxDepth, 
                    maxBins=best_maxBins, 
                    maxIter=best_maxIter,
                    seed=seed_value)

search_time_start = time.time()
pipeline_gbt = Pipeline(stages=[gbt])
pipelineModel_gbt_hyperopt = pipeline_gbt.fit(trainDF)
print('Time to fit best model:', time.time() - search_time_start)
pipelineModel_gbt_hyperopt.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/Hyperopt/pipelineModel_gbt_hyperopt_smote_f1')

2022/10/09 17:02:40 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd43b05a38df644b8aca7e5b2216083f2', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/09 17:15:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpq31djqta/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to fit best model: 762.629944562912


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hyperopt.transform(testDF)

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('Hyperopt: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

Hyperopt: Gradient Boosted Trees
Area under ROC curve: 0.986733
Test Error: 0.0132665 
Accuracy: 0.986170234904838
